# Density

### In this notebook we make the features. For this we count all the particles in a cube, centered by a particle which is considered to be in a halo or not in future. This is done for different cube sizes. 


In [1]:
import pandas as pd
import numpy as np  
#box size
L=62.5
#the middle box size:
r_max = 10

In [4]:

df =  pd.read_csv(r'/Users/erfan/Desktop/particles_information.csv')
print(len(df))
#Sorting the data frame by x location
df = df.sort_values( by =['x'])

print(df)

19683000
          snapnum  phkey             x          y          z          vx  \
423750          0    707  5.066395e-07  13.195392  21.925102 -415.080350   
10266766        0  32040  3.814697e-06   9.751561  44.053860   49.972237   
10231065        0  32195  7.629394e-06   2.230024  40.391212  130.391390   
12967545        0  27561  1.144409e-05  62.060127  40.056350  186.225600   
16190820        0  26951  1.146179e-05  36.668083  46.438698  -52.547157   
273842          0    457  1.287460e-05   7.383055   8.321121 -369.892030   
17655133        0  29392  1.770537e-05  19.740864  44.136180  -41.704850   
3106693         0   5173  2.119690e-05  52.639587  21.552774 -134.508560   
43511           0     73  2.173334e-05  14.651722   0.120123 -515.170960   
2138047         0   3558  2.497807e-05  24.314573  16.112642  -69.536070   
16334649        0  27191  2.943724e-05  53.549942  36.597584  -51.103080   
17655143        0  29392  4.220568e-05  19.930727  44.770230 -100.726910   
176

In [5]:
#periodic boundry condition

d=df[df.x > r_max]

d=d[d.x < L-r_max]

d=d[d.y > r_max]

d=d[d.y < L-r_max]

d=d[d.z > r_max]

d=d[d.z < L-r_max]


print(d)

          snapnum  phkey          x          y          z          vx  \
3542133         0   5899  10.000003  43.598644  16.648273  -16.022896   
19459156        0  32394  10.000004  14.992266  47.942020  -54.246430   
461407          0    770  10.000043  13.058078  21.863415 -233.205900   
222439          0    371  10.000044  12.835334  12.342678 -285.898470   
3552037         0   5916  10.000046  41.758327  19.871122  111.870430   
17402490        0  28972  10.000053  27.633457  49.448880 -227.064220   
16483424        0  27439  10.000060  49.137695  45.729780 -128.429410   
17770600        0  29584  10.000060  25.193302  33.988457  128.999600   
1986463         0   3306  10.000064  22.712078  16.101095 -259.047300   
3622061         0   6032  10.000082  34.629660  24.906282  -18.829971   
1895903         0   3155  10.000086  27.464243  28.655922 -135.335270   
16022770        0  26672  10.000095  40.331180  34.788795  -63.740250   
1939122         0   3227  10.000103  21.743223  26.

In [6]:
l = len(df)
#Converting df to a numpy array to optimizing the velocity
adf = np.array(df['x'])
adf = np.append(adf , np.array(df['y']) )
adf = np.append(adf , np.array(df['z']) )
adf = np.append(adf , np.array(df['id']) ).reshape(4,l)
adf = np.transpose(adf)

print(adf)

[[5.0663950e-07 1.3195392e+01 2.1925102e+01 4.2375100e+05]
 [3.8146973e-06 9.7515610e+00 4.4053860e+01 1.0266767e+07]
 [7.6293945e-06 2.2300240e+00 4.0391212e+01 1.0231066e+07]
 ...
 [6.2499996e+01 5.0952473e+01 4.6083537e-01 2.8051810e+06]
 [6.2499996e+01 4.2445374e+01 1.7876396e+01 3.4994660e+06]
 [6.2499996e+01 3.6515900e+01 4.1774520e+01 1.6220599e+07]]


In [7]:
import datetime

#Function for calculating density with top-hat function
def density(r_cutof ,df , d):
    #s=datetime.datetime.now().replace(microsecond=0)
    m=len(d.index)
    den=pd.DataFrame(np.zeros(m),columns=['density'])
    i = 0
    while(i!=m):
        wx=d.iloc[i]['x']
        wy=d.iloc[i]['y']
        wz=d.iloc[i]['z']
        ix = np.searchsorted(adf[:,0] , wx-r_cutof)
        jx = np.searchsorted(adf[:,0] , wx+r_cutof)
        a=np.where( (adf[ix:jx,1] < wy+r_cutof) & (adf[ix:jx,1] > wy-r_cutof) & (adf[ix:jx,2] < wz+r_cutof) & (adf[ix:jx,2] > wz-r_cutof) )
    
        den.density[i]=len(a[0])
    
        i = i+1
    return den
print('Data successfully saved, functions are formed')

Data successfully saved, functions are formed


In [22]:
print(len(d))
print(len(adf))

5161764
19683000


In [23]:
# selecting n random particles to tracing their tragectories 
n = 100000
sample = d.sample(n , random_state = 1)
print(sample)
sample_density = sample[['id','In_halo']]
sample_density.reset_index(drop=True, inplace=True)
print(sample_density)
rc = np.array([])

for i in range(8 , 11 , 1):
    r = 0.8 + 0.4 * i
    rc = np.append(rc,r)
    print('the cut off is:{}' .format(rc))
    den = density(r , adf , sample )
    #converting the number of particles to density
    den = den.div((2*r)**3)
    print(i)
    sample_density = pd.concat([sample_density, den ] ,axis = 1)
    sample_density = sample_density.rename(columns={'density': 'den_rc={}'.format(r)})
    #print(sample_density)
    
print(sample_density)
print('done')

          snapnum  phkey          x          y          z          vx  \
10812779        0  18000  33.072178  27.753729  36.802340   55.710743   
9038600         0  15047  31.869438  10.151769  16.056200  527.884500   
8843546         0  14723  37.933067  25.008278  28.903440   34.017303   
17913417        0  29821  17.601572  26.865538  38.017254   92.238760   
13667886        0  22755  41.162970  42.802597  41.313755 -432.837620   
1918921         0   3194  12.897427  26.174726  26.876553  -31.580446   
8797884         0  14647  46.132042  18.680593  30.211640  -80.688170   
5682192         0   9459  44.585400  49.418530  21.239784 -109.799660   
9077503         0  15112  40.574627  12.284349  17.552195 -192.353960   
6067423         0  10099  44.248188  34.890270  20.640495 -104.307900   
1754986         0   2922  22.544050  21.012728  29.940546  188.150340   
10817881        0  18008  32.411762  26.187140  38.637890   -3.500413   
4768800         0   7939  23.577034  37.952694  13.

8
the cut off is:[4.  4.4]
9
the cut off is:[4.  4.4 4.8]
10
             id  In_halo  density_rc=4.0  density_rc=4.4  density_rc=4.8
0      10812780        0       79.955078       80.150028       80.232973
1       9038601        0       80.167969       80.368673       80.469202
2       8843547        0       80.886719       80.817701       80.826371
3      17913418        0       80.732422       80.666557       80.578839
4      13667887        0       79.859375       79.868285       79.991093
5       1918922        1       80.263672       80.326118       80.481635
6       8797885        1       80.958984       81.021671       80.993652
7       5682193        0       80.947266       81.061291       81.228751
8       9077504        0       81.246094       81.256457       81.269441
9       6067424        0       81.423828       81.479503       81.390381
10      1754987        1       81.179688       81.147868       81.217448
11     10817882        0       80.691406       80.679764       

In [24]:
print(len(adf))
#np.savetxt('Prticle_density_rc_2.8/3.6.csv', sample_density)
sample_density.to_csv('/Users/erfan/Desktop/Prticle_density_rc_4to5.csv',index=False)
print('done')

19683000
done


In [10]:
print(sample_density)

             id  In_halo   den_rc=4  den_rc=4.4  den_rc=4.8
0      10812780        0  79.955078   80.150028   80.232973
1       9038601        0  80.167969   80.368673   80.469202
2       8843547        0  80.886719   80.817701   80.826371
3      17913418        0  80.732422   80.666557   80.578839
4      13667887        0  79.859375   79.868285   79.991093
5       1918922        1  80.263672   80.326118   80.481635
6       8797885        1  80.958984   81.021671   80.993652
7       5682193        0  80.947266   81.061291   81.228751
8       9077504        0  81.246094   81.256457   81.269441
9       6067424        0  81.423828   81.479503   81.390381
10      1754987        1  81.179688   81.147868   81.217448
11     10817882        0  80.691406   80.679764   80.574318
12      4768801        0  79.386719   79.652576   79.781991
13     12407878        0  79.980469   80.056114   80.132378
14     15936274        0  81.187500   81.049552   80.893057
15      8873289        0  79.521484   79

### Also density is converted to density-contrast by:     
### $\delta =(ρ-ρ_0)/ρ_0$ 

### where $ρ$ is the cube density and $ρ_0$ is the background density ( 19683000 particles in a 62.5* 62.5* 62.5 cube )

In [11]:
p=19683000/(62.5**3)
sample_density.iloc[:,2:] -= p
sample_density.iloc[:,2:] /= p
print(sample_density)

             id  In_halo  den_rc=4  den_rc=4.4  den_rc=4.8
0      10812780        0 -0.008267   -0.005849   -0.004820
1       9038601        0 -0.005626   -0.003137   -0.001890
2       8843547        0  0.003289    0.002433    0.002540
3      17913418        0  0.001375    0.000558   -0.000530
4      13667887        0 -0.009454   -0.009343   -0.007820
5       1918922        1 -0.004439   -0.003665   -0.001736
6       8797885        1  0.004185    0.004963    0.004615
7       5682193        0  0.004040    0.005454    0.007531
8       9077504        0  0.007746    0.007875    0.008036
9       6067424        0  0.009951    0.010642    0.009536
10      1754987        1  0.006923    0.006528    0.007391
11     10817882        0  0.000866    0.000722   -0.000586
12      4768801        0 -0.015317   -0.012019   -0.010414
13     12407878        0 -0.007952   -0.007014   -0.006068
14     15936274        0  0.007020    0.005309    0.003367
15      8873289        0 -0.013645   -0.011855   -0.0101

In [12]:
sample_density.head()

,id,In_halo,den_rc=4,den_rc=4.4,den_rc=4.8
0,10812780,0,-0.008267,-0.005849,-0.00482
1,9038601,0,-0.005626,-0.003137,-0.00189
2,8843547,0,0.003289,0.002433,0.00254
3,17913418,0,0.001375,0.000558,-0.00053
4,13667887,0,-0.009454,-0.009343,-0.00782
